# LMNA

In [1]:
import genophenocorr
print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


In [2]:
import hpotk

fpath_hpo = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2024-02-08/hp.json'
hpo = hpotk.load_minimal_ontology(fpath_hpo)

In [3]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'input/'
cohort_creator = configure_caching_cohort_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

Patients Created: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 127/127 [00:00<00:00, 414.46it/s]
Validated under none policy
No errors or warnings were found


## Transcript
Get it by looking for gene symbol in https://www.ncbi.nlm.nih.gov/clinvar/ look for pathogenic
zcopy it and go to https://variantvalidator.org/service/validate/, then scroll down to copy GRCH38 string, copy this and put into same website

looking for MANE select plus clinical (most important transcript for clinic) if doesnt exist, use clinvar result

In [4]:
tx_id = 'NM_170707.4'

## Explore Cohort

In [5]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewable

viewer = CohortViewable(hpo, transcript_id=tx_id)

In [6]:
display(HTML(viewer.process(cohort)))

HPO Term,ID,Annotation Count
Ankle contracture,HP:0034677,39
Elbow contracture,HP:0034391,37
Spinal rigidity,HP:0003306,37
Stiff neck,HP:0025258,36
Proximal muscle weakness in upper limbs,HP:0008997,35
Dilated cardiomyopathy,HP:0001644,27
Atrial fibrillation,HP:0005110,25
Arrhythmia,HP:0011675,23
Lipodystrophy,HP:0009125,21
Distal lower limb muscle weakness,HP:0009053,21


In [7]:
from genophenocorr.view import DiseaseViewable

dis_view = DiseaseViewable(hpo, transcript_id=tx_id)
dis_dict = dis_view._prepare_context(cohort)
display(HTML(dis_view.process(cohort)))

## Configure the analysis

In [8]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration

analysis_config = CohortAnalysisConfiguration()
analysis_config.missing_implies_excluded = True
analysis_config.pval_correction = 'fdr_bh'

analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)


In [9]:
from genophenocorr.model import VariantEffect
from genophenocorr.analysis.predicate import PatientCategories
from genophenocorr.analysis.predicate.genotype import VariantEffectPredicate

by_missense = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=tx_id)
by_missense.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_170707.4                       Yes             No  \
                                                    Count Percent  Count   
Lipodystrophy [HP:0009125]                          10/75     13%  15/35   
Micrognathia [HP:0000347]                             0/3      0%  15/15   
Prominent superficial blood vessels [HP:0007394]      0/3      0%  15/15   
Alopecia [HP:0001596]                                 0/3      0%  15/15   
Failure to thrive [HP:0001508]                        0/3      0%  15/15   
...                                                   ...     ...    ...   
Abnormality of the Achilles tendon [HP:0005109]       3/3    100%    2/2   
Abnormal tissue metabolite concentration [HP:00...    4/4    100%    0/0   
Proximal muscle weakness [HP:0003701]               44/44    100%    9/9   
Poor head control [HP:0002421]                      13/13    100%    2/2   
Abnormality of globe location [HP:0100886]            0/0      0%  15/15   

MISSENSE_VARIANT on NM_170707.4                                       \
                                                   Percent   p value   
Lipodystrophy [HP:0009125]                             43%  0.001167   
Micrognathia [HP:0000347]                             100%  0.001225   
Prominent superficial blood vessels [HP:0007394]      100%  0.001225   
Alopecia [HP:0001596]                                 100%  0.001225   
Failure to thrive [HP:0001508]                        100%  0.001225   
...                                                    ...       ...   
Abnormality of the Achilles tendon [HP:0005109]       100%  1.000000   
Abnormal tissue metabolite concentration [HP:00...      0%  1.000000   
Proximal muscle weakness [HP:0003701]                 100%  1.000000   
Poor head control [HP:0002421]                        100%  1.000000   
Abnormality of globe location [HP:0100886]            100%  1.000000   

MISSENSE_VARIANT on NM_170707.4                                       
                                                   Corrected p value  
Lipodystrophy [HP:0009125]                                  0.079657  
Micrognathia [HP:0000347]                                   0.079657  
Prominent superficial blood vessels [HP:0007394]            0.079657  
Alopecia [HP:0001596]                                       0.079657  
Failure to thrive [HP:0001508]                              0.079657  
...                                                              ...  
Abnormality of the Achilles tendon [HP:0005109]             1.000000  
Abnormal tissue metabolite concentration [HP:00...          1.000000  
Proximal muscle weakness [HP:0003701]                       1.000000  
Poor head control [HP:0002421]                              1.000000  
Abnormality of globe location [HP:0100886]                  1.000000  

[455 rows x 6 columns]

In [10]:
by_dis = analysis.compare_disease_vs_genotype(VariantEffectPredicate(transcript_id=tx_id, effect=VariantEffect.MISSENSE_VARIANT))
by_dis.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_170707.4    Yes             No                        \
                                 Count Percent  Count Percent       p value   
OMIM:176670                       0/90      0%  15/37     41%  8.025624e-10   
OMIM:151660                      12/90     13%   0/37      0%  1.807470e-02   
OMIM:181350                      34/90     38%   7/37     19%  5.904112e-02   
OMIM:613205                      13/90     14%   2/37      5%  2.276691e-01   
OMIM:115200                      31/90     34%  13/37     35%  1.000000e+00   

MISSENSE_VARIANT on NM_170707.4                    
                                Corrected p value  
OMIM:176670                          4.012812e-09  
OMIM:151660                          4.518674e-02  
OMIM:181350                          9.840187e-02  
OMIM:613205                          2.845863e-01  
OMIM:115200                          1.000000e+00